In [ ]:
import numpy as np
import librosa

ff = "wavs_single_channel/BoundaryTone-features-pyin/subj-2108_piano_1obj_mbt_zen.wav_1.wav.npy"
# load features from npy
features = np.load(ff)

In [ ]:
# get first non-zero frame
non_zero_frames = np.where(features[1] != 0)
first_non_zero_frame = non_zero_frames[0][0]
y, sr = librosa.load('wavs_single_channel/BoundaryTone/subj-2108_piano_1obj_mbt_zen.wav_1.wav', sr=44100)

print(y.shape, sr)
print(y.shape[0]/44100)
f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                            sr=16000,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(voiced_flag.shape)

f02, voiced_flag2, voiced_probs2 = librosa.pyin(y,
                                            sr=44100,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(voiced_flag2[-3:])
# print(features.shape)
# print(features.shape[1]/44100)



In [66]:
import webrtcvad



def get_vad(y, sr, mode=2):
    vad = webrtcvad.Vad()
    vad.set_mode(mode)

    sample_rate = 16000
    # downsampling from 44100 to 16000 and framing
    y_16k = librosa.resample(y, orig_sr=sr, target_sr=sample_rate)
    # transfer to bytes
    y_16k = np.int16(y_16k * 32768).tobytes()
    # calculate bytes of 10ms in 16000Hz
    step = int(sample_rate * 0.01) * 2

    # assert error if len(y_16k) < step
    assert len(y_16k) >= step

    # loop the bytes in 10ms frames
    frames = []
    for i in range(0, len(y_16k) - step, step):
        frames.append(y_16k[i:i+step])

    vadres = []
    for i, frame in enumerate(frames):
        vadres.append(vad.is_speech(frame, sample_rate))
    return vadres

print(get_vad(y, sr, mode=1))


[False, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, T